1. DATA COLLECTING

In [ ]:
import pandas as pd
import seaborn as sns

dataset = sns.load_dataset('titanic')
dataset.to_csv('Titanic-Dataset.csv')
dataset

2. HANDLING MISSING VALUES

In [ ]:
dataset.isna().sum()

AS THERE ARE ABOUT 80% MISSING VALUES IN DECK LETS REMOVE THE COLUMNS AS IT IS POOR DATA

In [ ]:
dataset.drop('deck', axis = 1,inplace = True)
dataset

NOW FILLING AGE COLUMN, FILL WITH MEAN AS THE DATA IS NORMALLY DISTRIBUTED

In [ ]:
dataset['age'] = dataset['age'].fillna(dataset['age'].mean())
dataset

REMOVING THE REMAINING MISSING VALUES

In [ ]:
dataset = dataset.dropna().reset_index(drop=True)
dataset.isnull().sum()

3. REMOVE DUPLICATES

AS THERE ARE NO NAMES OR UNIQUE ID's TAKING SOME PARAMETERS THAT ENSURES THE COLUMN IS DUPLICATE

In [ ]:
dataset.duplicated().sum()

In [ ]:
dataset

In [ ]:
dataset = dataset.drop_duplicates().reset_index(drop=True)
dataset.duplicated().sum()

ALSO THERE ARE DUPLICATE COLUMNS SO REMOVE THAT TOO

In [ ]:
#alone column also irrelevant as aleady checking spouse and family
dataset.drop(['who','alive','embarked','alone'],axis=1,inplace =True)

4. TYPE CONVERSION

In [ ]:
dataset.info()

In [ ]:
dataset[['sex','embark_town']] = dataset[['sex','embark_town']].astype('category')
dataset[['age','fare']] = dataset[['age','fare']].astype('int')
dataset['adult_male'] = dataset['adult_male'].astype('int')
dataset

HANDLE CATEGORICAL DATA

ORDINAL ENCODING FOR DATA THAT HAS A RELATION OR IMPLICIT ORDERING

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

set_category=[['Third' , 'Second' , 'First']]
orden = OrdinalEncoder(categories=set_category)
dataset['class'] = orden.fit_transform(dataset[['class']]).astype('int')
dataset

ONE-HOT ENCODING FOR REST OF THE CATEGORICAL FEATURES

In [ ]:
from sklearn.preprocessing import OneHotEncoder

hoten = OneHotEncoder(sparse_output=False)
encoded = hoten.fit_transform(dataset[['sex']])
featured_cols = hoten.get_feature_names_out(['sex'])
df = pd.DataFrame(encoded , columns=featured_cols)
dataset = pd.concat([dataset,df],axis=1)
dataset = dataset.drop(['sex'] , axis = 1)
dataset

LABEL ENCODING 

In [ ]:
from sklearn.preprocessing import LabelEncoder

laben = LabelEncoder()
dataset['embark_town'] = laben.fit_transform(dataset['embark_town'])
dataset

VISUALIZATION USING SEABORN

USE HISTPLOT FOR CHECKING TYPE OF DISTRIBUTION

In [ ]:
import matplotlib.pyplot as plt

for i in dataset:
    sns.histplot(data=dataset,x=i , color='g')
    plt.show()

BOXPLOT FOR RELATIONSHIP AND DETECTING OUTLIERS

In [ ]:
#check relationships
plt.figure(figsize=(18,3))
sns.boxplot(data=dataset, x = 'age' , y = 'survived')

In [ ]:
plt.figure(figsize=(18, 3))
sns.boxplot(data=dataset, x='class',y='age', hue='survived', palette='Set3')

In [ ]:
dataset.groupby('survived')['class'].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=dataset, x='class', hue='survived', palette='Set3',multiple='dodge')
plt.title('Count of Passengers by Class and Survival Status')
plt.xlabel('Passenger Class')
plt.ylabel('Count')
plt.legend(title='Survived', labels=['No', 'Yes'])
plt.show()


In [ ]:
plt.figure(figsize=(18,3))
sns.boxplot(data=dataset, x = 'sex_male' , y = 'survived' , color='b')

In [ ]:
#detect outliers

for i in dataset:
    sns.boxplot(data=dataset , x=i )
    plt.show()

FROM VISUALIZATION UNDERSTOOD THAT SOME OF THE COLUMNS CONTAINS OUTLIERS 

THEY ARE : AGE , SIBSP , PARCH , FARE

BUT FROM DOMAIN KNOWLEDGE CAN UNDERSTAND THAT THERE ARE NO OUTLIERS IN SISBSP AND PARCH

In [ ]:
dataset

OUTLIERS REMOVAL

In [ ]:
#REMOVE OUTLIERS IN AGE

upper = dataset['age'].quantile(0.95)
lower = dataset['age'].quantile(0.05)

dataset['age'] = dataset['age'].clip(lower,upper)
dataset

In [ ]:
#REMOVE OUTLIERS IN FARE

Q3 = dataset['fare'].quantile(0.75)
Q1 = dataset['fare'].quantile(0.25)

IQR = Q3-Q1

upper = Q3 + ( 1.5 * IQR)
lower = Q1 - ( 1.5 * IQR)

dataset['fare'] = dataset['fare'].clip(lower,upper)
dataset

SPLITTING TRAIN AND TEST DATA

FEATURE TRANSFORMATION AND SCALING ( PERFORMING SEPERATELY TO AVOID DATA LEAKAGE )

MINMAX SCALING OR NORMALIZATION FOR AGE

In [ ]:
min = dataset['age'].min()
max = dataset['age'].max()

dataset['age'] = ( dataset['age'] - min ) / ( max - min )
dataset

STANDARDIZATION OR Z-SCORE SCALING FOR FARE

In [ ]:
mean = dataset['fare'].mean()
std = dataset['fare'].std()

dataset['fare'] = ( dataset['fare'] - mean ) / std
dataset

TEST SET

MINMAX SCALING OR NORMALIZATION FOR AGE ( USING SCIKIT-LEARN )

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scale = MinMaxScaler()
dataset['age'] = scale.fit_transform(dataset[['age']])
dataset


STANDARDIZATION OR Z-SCORE SCALING FOR FARE

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
dataset['fare'] = scaler.fit_transform(dataset[['fare']])
dataset

In [ ]:
x=dataset.drop('fare',axis=1)
x.head(10)

In [ ]:
y=dataset.fare
y.head(10)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression 
lnr_C=LinearRegression()
lnr_C.fit(X_test,y_test)
lnr_C.score(X_train,y_train)

In [ ]:

from sklearn.metrics import mean_absolute_error
pred = lnr_C.predict(X_test)
mean_absolute_error(pred,y_test)